In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from google.colab import drive
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

In [6]:
drive.mount('/content/drive')
dataset_path = "/content/drive/MyDrive/image_data/dataset"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(),  # Data Augmentation
    transforms.RandomRotation(10),  # Generalizes better
    transforms.ToTensor(),     # Convert image to pytorch tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization
    #Centers data around 0, leads to faster convergence
])

In [8]:
#extracting handcrafted features
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
X, y = [], []
for img, label in dataset:
    img_np = np.array(img.permute(1, 2, 0) * 255, dtype=np.uint8)  # Convert to NumPy

    # Extract HOG features
    hog_features = hog(img_np, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                       orientations=9, channel_axis=-1)

    # Extract color histogram features
    hist_features = cv2.calcHist([img_np], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist_features = cv2.normalize(hist_features, hist_features).flatten()

    # Concatenate features
    feature_vector = np.hstack((hog_features, hist_features))
    X.append(feature_vector)
    y.append(label)

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [9]:
#train test split for random forest and svm
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Hyperparameter tuning for SVM
svm_param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['scale', 'auto']}
svm_clf = RandomizedSearchCV(SVC(), svm_param_grid, n_iter=10, cv=3, random_state=42)
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)
svm_acc = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {svm_acc * 100:.2f}%")

SVM Accuracy: 91.45%


In [11]:
# Hyperparameter tuning for Random Forest
rf_param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
rf_clf = RandomizedSearchCV(RandomForestClassifier(random_state=42), rf_param_grid, n_iter=10, cv=3, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
rf_acc = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {rf_acc * 100:.2f}%")

Random Forest Accuracy: 88.89%


In [12]:
# train-test split for cnn
full_data = datasets.ImageFolder(root=dataset_path, transform=transform) # Automatically assigns label based on folder name
train_size = int(0.8 * len(full_data))
test_size = len(full_data) - train_size
train_data, test_data = random_split(full_data, [train_size, test_size])

In [13]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)   # Makes mini-batches of size 32
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [14]:
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Flatten(),
    nn.Linear(64 * 8 * 8, 128), # 64 are the filters and 8*8 is the spatial dimension ( because of prev layers )
    nn.ReLU(),
    nn.Linear(128, 2)  # Binary classification (Mask / No Mask)
)
# Note: No. of output channels is a design choice
# Just one FC layer afte conv layers is enough more layers may introduce overfitting

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=4096, out_features=128, bias=True)
  (8): ReLU()
  (9): Linear(in_features=128, out_features=2, bias=True)
)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # Initial learning rate, adam optimizer automatically adjusts learning rate

In [17]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad() # reset gradients to ensure, results from prev iterations don't accumulate
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step() #updates weights

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10, Loss: 0.3930
Epoch 2/10, Loss: 0.2469
Epoch 3/10, Loss: 0.1739
Epoch 4/10, Loss: 0.1394
Epoch 5/10, Loss: 0.1178
Epoch 6/10, Loss: 0.0989
Epoch 7/10, Loss: 0.0940
Epoch 8/10, Loss: 0.0764
Epoch 9/10, Loss: 0.0714
Epoch 10/10, Loss: 0.0647


In [18]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
cnn_acc = 100 * correct / total
print(f" Test Accuracy: {100 * correct / total:.2f}%")

torch.save(model.state_dict(), "/content/drive/MyDrive/mask_model.pth")

 Test Accuracy: 95.73%


In [19]:
# MLP model
mlp = nn.Sequential(
    nn.Linear(len(X_train[0]), 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 2)
)
mlp.to(device)
mlp_criterion = nn.CrossEntropyLoss()
mlp_optimizer = optim.Adam(mlp.parameters(), lr=0.001)


In [20]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

<ipython-input-20-6c5cfcaf2222>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)


In [21]:
num_epochs = 10
for epoch in range(num_epochs):
    mlp.train()
    mlp_optimizer.zero_grad()
    outputs = mlp(X_train_tensor)
    loss = mlp_criterion(outputs, y_train_tensor)
    loss.backward()
    mlp_optimizer.step()
    print(f"MLP Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

mlp.eval()
with torch.no_grad():
    mlp_outputs = mlp(X_test_tensor)
    _, mlp_predicted = torch.max(mlp_outputs, 1)
mlp_acc = accuracy_score(y_test, mlp_predicted.cpu().numpy()) * 100


MLP Epoch 1/10, Loss: 0.6943
MLP Epoch 2/10, Loss: 0.6902
MLP Epoch 3/10, Loss: 0.6861
MLP Epoch 4/10, Loss: 0.6817
MLP Epoch 5/10, Loss: 0.6762
MLP Epoch 6/10, Loss: 0.6695
MLP Epoch 7/10, Loss: 0.6619
MLP Epoch 8/10, Loss: 0.6531
MLP Epoch 9/10, Loss: 0.6432
MLP Epoch 10/10, Loss: 0.6321


In [22]:
# Compare Results
print("\nComparison:")
print(f"SVM Accuracy: {svm_acc * 100:.2f}%")
print(f"Random Forest Accuracy: {rf_acc * 100:.2f}%")
print(f"CNN Accuracy: {cnn_acc:.2f}%")
print(f"MLP Accuracy: {mlp_acc:.2f}%")


Comparison:
SVM Accuracy: 91.45%
Random Forest Accuracy: 88.89%
CNN Accuracy: 95.73%
MLP Accuracy: 80.95%
